# Daily Retreat: Using Sentiment Analysis to<br>Find, Personalize, and Share Positive News from Popular Online Sources
__Aaron Carr, Azucena Faus, and Dave Friesen - ADS-599-01-SU23__

# Final EDA

* Import Libraries

* Define Functions for EDA

* Load Cleaned/tokenized data

* EDA  
    * Format publish_date for time-based visualizations
    * Find nulls/missing data
    * Descriptive Statistics
    * Word Cloud for entire corpus
    * Topic Modeling using Non-Negative Matrix Factorization
    * Topic Modeling using Latent Dirichlet Allocation
    * Coherence scores for both methods
    * Coherence vs number of Topics LDA
    * Coherence vs number of Topics NMF
    * NMF Word Cloud
    * LDA Word Cloud
    * Topical Distribution per Day Trends charts
    * Topical Distribution of entire Corpus
    * Closer look at the Amazon Reviews topic in the corpus

## Globally import libraries

In [1]:
#! pip install pyLDavis

import numpy as np
import pandas as pd

# Text preprocessing libraries
import requests
import os
import shutil
import re
import logging
import time
import zipfile
import requests
from bs4 import BeautifulSoup
import datetime
import re
import regex as rex
from bs4 import BeautifulSoup
import datetime
import json
from string import punctuation
from nltk.corpus import stopwords
import re
import emoji
import itertools
import pickle

# EDA for text libraries
import nltk
import collections
import textstat
from textstat import textstat
legacy_round = textstat._legacy_round
lang = 'en'
textstat.set_lang(lang)
from collections import defaultdict, Counter
from gensim.corpora import Dictionary
import random
#import textacy.preprocessing as tprep
#from textacy.extract import keyword_in_context
import pyLDAvis
pyLDAvis.enable_notebook()
from tqdm.auto import tqdm
#import spacy
import pyLDAvis.lda_model
import pyLDAvis.gensim_models
from wordcloud import WordCloud 
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF
from sklearn.pipeline import Pipeline
from sklearn.decomposition import LatentDirichletAllocation
from gensim import models, interfaces, utils
from gensim.models import Nmf
from gensim.models.coherencemodel import CoherenceModel
from gensim.models import TfidfModel
from gensim.models import LdaModel
from gensim.models.ldamulticore import LdaMulticore

# visuals
import matplotlib.pyplot as plt
import seaborn as sns
import calendar 
from tabulate import tabulate

# modeling
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_recall_curve
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import random
import statsmodels.tools.tools as stattools
from sklearn.tree import plot_tree
from sklearn.metrics import confusion_matrix
from tabulate import tabulate
from sklearn.ensemble import RandomForestClassifier
from scipy import stats
import seaborn as sns
import plotly.express as px
from sklearn  import metrics
from sklearn.datasets import make_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import shuffle


import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

ModuleNotFoundError: No module named 'textstat'

## Functions:

### Data pre-processing:

REGEX and NORMALIZE FUNCTIONS

In [ ]:
rex_sep = rex.compile(r'&nbsp;')
rex_ucode = rex.compile(r'[\\]u20*')

'''re.sub lambda citation:
https://chat.openai.com/share/402ec66e-2802-4cda-af8c-6f9f5b097d85
'''
sep_lst = []
ucode_lst = []
# Add leading and trailing space to URLs
def rex_replace(text):
    #txt = str(text)
    #print(lambda x: x.replace('&nbsp;', ' '))
    #sep_lst.append(rex_sep.findall(txt))
    #ucode_lst.append(rex_ucode.findall(txt))
    text = text.replace(r'&nbsp;', ' ').replace(r'-', ' ')\
    .replace(r'\n', ' ').replace('\u2063', ' ').replace('\u2066', ' ')\
    .replace('\u2069', ' ').replace('\u200b', ' ').replace('\u200d', ' ')\
    .replace('(click to view)', ' ')\
    .replace('a post shared by', ' ')\
    .replace('app users click here', ' ')\
    .replace('app users: click here', ' ')\
    .replace('app users, click here:', ' ')\
    .replace('click here.', ' ')\
    .replace('click here for more cartoons', ' ')\
    .replace('click here for more', ' ')\
    .replace('click here for more sports coverage on foxnews.com', ' ')\
    .replace('click here for other fox news digital adoptable pets stories', ' ')\
    .replace('click here for the fox news app', ' ')\
    .replace('click here for the latest fox news reporting', ' ')\
    .replace('click here for topline and cross tabs conducted', ' ')\
    .replace('click here to hear more', ' ')\
    .replace('click here to ge the fox news app', ' ')\
    .replace('click here to get the fox news app', ' ')\
    .replace('click here to get the opinion newsletter', ' ')\
    .replace('click here to learn more', ' ')\
    .replace('click here to read more', ' ')\
    .replace('click here to sign up for our health newsletter', ' ')\
    .replace('click here to sign up for our lifestyle newsletter', ' ')\
    .replace('click here to sign up for our opinion newsletter', ' ')\
    .replace('click here to sign up for the entertainment newsletter', ' ')\
    .replace('click here to subscribe and get your first year of fox nation free of charge', ' ')\
    .replace('click here to view', ' ')\
    .replace("click to get kurt's cyberguy newsletter with quick tips, tech reviews, security alerts and easy how to's to make you smarter", ' ')\
    .replace("click to get kurt's cyberguy newsletter with security alerts, quick tips, tech reviews, security and easy how to's to make you smarter", ' ')\
    .replace("click to get kurt's free cyberguy newsletter with quick tips, tech reviews, security alerts and easy how to's to make you smarter", ' ')\
    .replace("click to get kurt's free cyberguy newsletter with security alerts, quick tips, tech reviews, and easy how to's to make you smarter", ' ')\
    .replace('click to get the fox news app', ' ')\
    .replace('fox news digital', ' ')\
    .replace('request for comment', ' ')\
    .replace('the ap ', ' ')\
    .replace('copyright © 2023 breitbart', ' ')\
    .replace('all rights reserved', ' ')\
    .replace('copyright 2023 cyberguy.com', ' ')\
    .replace('copyright 2023 fox news network', ' ')\
    .replace('copyright 2023 viq media transcription', ' ')\
    .replace("please let us know if you're having issues with commenting", ' ')\
    .replace('view this post on instagram', ' ')
    #txt = txt
    #text = text.replace(r'200b', 'd171c')
    #text = rex_ucode.sub('', text)
    return text

def normalize(text):
    text = tprep.normalize.hyphenated_words(text)
    text = tprep.normalize.quotation_marks(text)
    text = tprep.normalize.unicode(text)
    text = tprep.remove.accents(text)
    return text

LEMMATIZATION FUNCTION

In [ ]:
#nlp_trans = spacy.load('en_core_web_sm')

#def lemma(text):
#    trans_txt = nlp_trans(text)
#    tokens = [t.lemma_ for t in trans_txt]
#    return tokens

* CASE LOAD 
* STOPWORD REMOVAL 
* URL REMOVAL 
* EMOJI REMOVAL 
* PUNCTUATION REMOVAL 
* MESSY TEXT REMOVAL
* TOKENIZE

In [ ]:
# CASE LOAD, REMOVE STOPWORDS,
# EMOJI and PUNCTUATION REMAL,
# URL REMOVAL
# TOKENIZE
# REMOVE MESSY text

punctuation = set(punctuation) # speeds up comparison
tw_punct = punctuation - {"#"}

sw = stopwords.words("english")
sw = sw + ['nan']
sw = sw + ['said'] + ['news'] + ['us'] + ['reuters'] + ['ap'] \
    + ['fox'] + ['cnn'] + ['breitbart'] + ['digital'] + ['follow'] \
    + ['associated press'] + ['press contributed'] + ['press'] \
    + ['dont'] + ['2023'] + ['told digital'] + ['associated contributed']\
    + ['contributed report'] + ['associated'] + ['contributed'] +\
    ['report'] + ['continued'] + ['reportedly'] + ['im']

# Two useful regex
whitespace_pattern = re.compile(r"\s+")
hashtag_pattern = re.compile(r"^#[0-9a-zA-Z]+")

def emoji_split(text):
    return("".join([' ' + ch + ' ' if emoji.is_emoji(ch) else ch for ch in text]))

def remove_stop(tokens) :
    # modify this function to remove stopwords

    return[t for t in tokens if t not in sw]
 
def remove_punctuation(text, punct_set=tw_punct) : 
    return("".join([ch for ch in text if ch not in punct_set]))

def tokenize(text) : 
    """ Splitting on whitespace rather than the book's tokenize function. That 
        function will drop tokens like '#hashtag' or '2A', which we need for Twitter. """
    
    return([item.lower() for item in whitespace_pattern.split(text)])
    
def remove_url(text):
    return(re.sub(r'http\S+', '', text))

def remove_messy(text): # remove words that give away the source
    text1=re.sub(r'cnn', '', text)
    text2=re.sub(r'fox', '', text1)
    text3=re.sub(r' — ', '', text2)
    text4=re.sub(r'breitbart', '', text3)
    return(re.sub(r'\\n', '', text4))

# two pipelines to either tokenize or simply remove punctuation
# and lowercase as we will need to extract feature words:

full_pipeline = [str.lower, remove_url, rex_replace, emoji_split, remove_messy, 
                 remove_punctuation, tokenize, remove_stop]
first_pipeline = [str.lower, remove_url, rex_replace, emoji_split, remove_messy, 
                  remove_punctuation]
tokenize_sw_pipeline = [remove_punctuation, remove_messy,tokenize, remove_stop]


def prepare(text, pipeline) : 
    tokens = str(text)
    
    for transform in pipeline : 
        tokens = transform(tokens)
        
    return(tokens)

### Feature extraction Function:

In [ ]:
def conv_features(text,fw) :
    feature_set = dict()
    for word in text.split():
        if word in fw:
            feature_set[word] = True
    return(feature_set)

### EDA functions:

* GET PATTERNS
* WORD COUNTS
* WORDCLOUD
* TOPIC MODELING

In [ ]:
def get_patterns(text_analyze, num_words, T):
    if(len(text_analyze)==0):
        raise ValueError("Can't work with empty text object")
    total_tokens = 1
    unique_tokens = 0
    avg_token_len = 0.0
    lexical_diversityP = 0.0
    top_words = []

    # Only applying the token_normal, which takes only alphanumeric values
    # to twitter data:
    if T == 1:
        text_analyze=token_normal(text_analyze)

    total_tokens = len(text_analyze)
    unique_tokens = len(set(text_analyze))
    lexical_diversityP = unique_tokens/total_tokens
    avg_token_len = np.mean([len(ta) for ta in text_analyze])

    top_words_1 = collections.Counter(text_analyze)
    top_words = top_words_1.most_common(num_words)

    results={'tokens': total_tokens,
             'unique_tokens': unique_tokens,
             'avg_token_length': avg_token_len,
             'lexical_diversity': lexical_diversityP,
             'top_words': top_words}
    return(results)

Word Cloud and Word Count Functions:

In [ ]:
def wordcloud(word_freq, title=None, max_words=200, stopwords=None):

    wc = WordCloud(width=800, height=400, 
                   background_color= "black", colormap="Paired", 
                   max_font_size=150, max_words=max_words)
    
    # convert data frame into dict
    if type(word_freq) == pd.Series:
        counter = Counter(word_freq.fillna(0).to_dict())
    else:
        counter = word_freq

    # filter stop words in frequency counter
    if stopwords is not None:
        
        counter = {token:freq for (token, freq) in counter.items() 
                              if token not in stopwords}
    wc.generate_from_frequencies(counter)
 
    plt.title(title) 

    plt.imshow(wc, interpolation='bilinear')
    plt.axis("off")
    
# Here, we only apply splitting to the lyrics data due to the difference
# in dataframe/data ingestion between twitter and lycis data:
    
#def count_words(df, column='tokens', preprocess=None, min_freq=2, split=0):
def count_words(x, preprocess=None, min_freq=2, split=0):

    # process tokens and update counter
    def update(doc):
        tokens = doc if preprocess is None else preprocess(doc)
        counter.update(tokens)

    # create counter and run through all data
    #counter = collections.Counter()
    #top_words_1 = collections.Counter(text_analyze)
    #top_words = top_words_1.most_common(num_words)
    if split == 0:
        counter = collections.Counter(x)
    else: 
        counter = collections.Counter(x.split())

    #df[column].map(update)

    # transform counter into data frame
    freq_df = pd.DataFrame.from_dict(counter, orient='index', columns=['freq'])
    freq_df = freq_df.query('freq >= @min_freq')
    freq_df.index.name = 'token'
    
    return freq_df.sort_values('freq', ascending=False)

Topic Modeling Display Function:

In [ ]:
def display_topics(model, features, no_top_words=5):
    for topic, words in enumerate(model.components_):
        total = words.sum()
        largest = words.argsort()[::-1] # invert sort order
        print("\nTopic %02d" % topic)
        for i in range(0, no_top_words):
            print("  %s (%2.2f)" % (features[largest[i]], 
                                    abs(words[largest[i]]*100.0/total)))

Topical Word Clouds Function:

In [ ]:
def wordcloud_topics(model, features, no_top_words=40):
    for topic, words in enumerate(model.components_):
        size = {}
        largest = words.argsort()[::-1]
        for i in range(0, no_top_words):
            size[features[largest[i]]] = abs(words[largest[i]])
        wc = WordCloud(background_color = 'white', max_words=100,
                    width=960, height=540)
        wc.generate_from_frequencies(size)
        plt.figure(figsize=(12,12))
        plt.imshow(wc,interpolation='bilinear')
        plt.axis("off")

---

## Load Data from CSV's:

**Full Dataset**

In [ ]:
news_data_df = pd.read_csv('../data_large/data_preprocessed_wo_sw_2023-07-20_13-02-01408354.csv')
news_data_w_sw_df = pd.read_csv('../data_large/data_preprocessed_w_sw_2023-07-20_13-02-01408354.csv')

In [ ]:
news_data_df.head()

In [ ]:
news_data_df.shape

---

## EDA


---

### Format Publish Date for Time-Based EDA charts:

In [ ]:
api_data_complete_df=news_data_df.copy()

In [ ]:
api_data_complete_df['publish_date'].value_counts()

In [ ]:
#api_data_complete_df['publish_date_day']=api_data_complete_df['publish_date'].str[0:10]

In [ ]:
api_data_complete_df['publish_date'].astype('string')

In [ ]:
#api_data_complete_df['publish_date_day']
new_format = "%Y-%m-%d"
for x in range(len(api_data_complete_df)):
    api_data_complete_df.loc[x,'publish_date'] = \
    pd.to_datetime(api_data_complete_df.loc[x, 'publish_date'],
                   format='%Y-%m-%dT%H:%M:%SZ').strftime(new_format)


In [ ]:
api_data_complete_df['publish_date'].head(6141)

----

### Missing Values:

Only one missing value for training data:

In [ ]:
api_data_complete_df.isna().sum()

In [ ]:
api_data_complete_df = api_data_complete_df.dropna(axis=0,
                                       subset=['processed_text']).reset_index()

---

**Most missing data is from the author feature which is not relevant to our analysis. There are 3 records with missing processed_text which is used but insignificantly affects our final word count/topic modeling/sentiment analyses.**

---

### Descriptive Statistics:

In [ ]:
api_data_complete_df['original_word_count'] = \
api_data_complete_df['article_text'].apply(lambda x: len(str(x).split(" ")))

In [ ]:
api_data_complete_df.tail()

In [ ]:
api_data_complete_df['num_tokens'].mean()

In [ ]:
api_data_complete_df.shape

In [ ]:
api_data_complete_df.describe()

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(10, 5))

# Plot the boxplot on the first subplot
sns.boxplot(x=api_data_complete_df['num_tokens'], ax=axs[0])

# Plot the histogram on the second subplot
sns.histplot(x=api_data_complete_df['num_tokens'], ax=axs[1])

# Show the plot
plt.show()

In [ ]:
# Boxplot plus histogram of token distribution to see outliers
sns.set(style="ticks")

x = np.random.randn(100)

f, (ax_box, ax_hist) = plt.subplots(2, sharex=True, 
                                    gridspec_kw={"height_ratios": (.25, .85)})
sns.boxplot(x=api_data_complete_df['num_tokens'], ax=ax_box)
sns.distplot(x=api_data_complete_df['num_tokens'], ax=ax_hist)
ax_box.set(title='Number of Tokens per Article Distribution')
ax_box.set_xlim([0, 4000])
ax_hist.set_xlim([0, 4000])

In [ ]:
api_data_complete_df['num_tokens'].hist(bins=70)
plt.title('Number of Tokens per Article Histogram')

In [ ]:
api_data_complete_df['num_tokens'].unique()

In [ ]:
api_data_complete_df.to_csv("../data_large/capstone_master_model_topic_input.csv",
                            sep=',')

---

Word counts had a right skew due to the very long length of 25% of articles. These articles were observed and deemed appropriate for inclusion in the dataset as they were not outliers, incorrect, or dirty data articles. The average, median, std deviation, 25th, 75th, minimum and maximum values are reported below.

![Descriptive Statistics on Word Counts per Article](../data/desc_stats.jpg)


---

### Readability of articles:

---

Readability scores are calculated for all the articles, for EDA purposes. Interesting things to note were the journals that tended to have higher school grade level reading like the Washington Post, Reuters, and ABC News.

The easiest reading level journals was BuzzFeed which had a 7th/8th grade reading level score.

---

In [ ]:
# Use data that includes stopwords for readability
# and remove nulls from processed_text feature

news_data_w_sw_df_2 = \
news_data_w_sw_df.dropna(axis=0, subset=['processed_text']).reset_index()

In [ ]:
# Calculate readability using various metrics on the
# processed text that includes stop words so it can
# determine ease of reading/undersatnding by grade level
# these scores are stored as features in the dataset

readability = []
dale_chall = []
reading_time = []
readability_overall = []
for row in news_data_w_sw_df_2['processed_text']:
    auto_readability_index_score = textstat.automated_readability_index(row)
    dale_chall_readability_score = textstat.dale_chall_readability_score(row)
    readingtime_score = textstat.reading_time(row, ms_per_char=14.69)
    readability.append(auto_readability_index_score)
    dale_chall.append(dale_chall_readability_score)
    reading_time.append(readingtime_score)

news_data_w_sw_df_2['auto_readability']=readability
news_data_w_sw_df_2['dale_chall_readability']=dale_chall
news_data_w_sw_df_2['reading_time']=reading_time

In [ ]:
news_data_w_sw_df_2['dale_chall_readability']

In [ ]:
news_data_w_sw_df['auto_readability'].mean()

In [ ]:
news_data_w_sw_df_2\
.groupby(news_data_w_sw_df_2['source_name']=="USA Today")['reading_time'].mean()

In [ ]:
news_data_w_sw_df_2.head()

In [ ]:
news_data_w_sw_df_3 = \
news_data_w_sw_df_2.dropna(axis=0, subset=['processed_text']).reset_index()

In [ ]:
news_data_w_sw_df_3['auto_readability'].mean()

In [ ]:
news_data_w_sw_df_3['dale_chall_readability'].mean()

In [ ]:
news_data_w_sw_df_3['reading_time'].mean()

In [ ]:
news_data_plot_reading_time = news_data_w_sw_df_3[['source_name','reading_time']]
news_data_plot_reading_time_avg = \
news_data_plot_reading_time\
.groupby(news_data_plot_reading_time['source_name'])['reading_time'].mean()

In [ ]:
news_data_plot_autoread = news_data_w_sw_df_3[['source_name','auto_readability']]
news_data_plot_autoread_avg = \
news_data_plot_autoread\
.groupby(news_data_plot_autoread['source_name'])['auto_readability'].mean()

In [ ]:
news_data_plot = news_data_w_sw_df_3[['source_name','dale_chall_readability']]
news_data_plot_avg = news_data_plot\
.groupby(news_data_plot['source_name'])['dale_chall_readability'].mean()

In [ ]:
#dale_chall readability per news source:
news_data_plot_avg

In [ ]:
ax_ar2 = news_data_plot_autoread_avg.plot(kind="line",
          x='source_name',
          rot=0,
          legend=True,
          figsize=(8,4),
          ylabel='School Grade Readabiity Score',
          xlabel='News Source',
          #='Political_Lean',
          title='School grade Readability Score average across News Sources')
#ax_ar2.bar_label(ax_ar2.containers[0])
plt.xticks(rotation=45)

In [ ]:
ax_ar = news_data_plot_autoread_avg.plot(kind="barh",
            y='source_name',
            rot=0,
            legend=True,
            figsize=(8,12),
            xlabel='School Grade Readabiity Score',
            ylabel='News Source',
            #='Political_Lean',
            title='School grade Readability Score average across News Sources')
ax_ar.bar_label(ax_ar.containers[0])

In [ ]:
ax_r = news_data_plot_reading_time_avg.plot(kind="barh",
                            y='source_name',
                            rot=0,
                            legend=True,
                            figsize=(8,12),
                            xlabel='Reading Time',
                            ylabel='News Source',
                            #='Political_Lean',
                            title='Reading Time average across News Sources')
ax_r.bar_label(ax_r.containers[0])

In [ ]:
#news_data_w_sw_df['auto_readability'].
#ax1=news_data_w_sw_df_average
#ax1=news_data_w_sw_df.groupby(news_data_w_sw_df['source_name'])['auto_readability']
ax1 = news_data_plot_avg.plot(kind="barh",
            y='source_name',
            rot=0,
            legend=True,
            figsize=(8,12),
            xlabel='Dale Chall Score',
            ylabel='News Source',
            #='Political_Lean',
            title='Dale Chall Readability Score average across News Sources')
ax1.bar_label(ax1.containers[0])

--------

### A brief look into the long-length "outlier-ISH" articles:

In [ ]:
#long_articles = api_data_complete_df.groupby(api_data_complete_df['num_tokens']>507)

long_articles_idx = \
api_data_complete_df.groupby(api_data_complete_df['num_tokens'] > 507)[['url',
                                                           'num_tokens',
                                                           'processed_text']]
long_articles_df = pd.DataFrame(long_articles_idx)
long_articles_df.head()
long_articles_df2 = pd.DataFrame(long_articles_df.iloc[1,1])

In [ ]:
long_articles_df2.shape

In [ ]:
long_articles_df2.describe()

In [ ]:
XTRAlong_articles_idx = \
api_data_complete_df.groupby(api_data_complete_df['num_tokens'] >1000)[['url',
                                                            'num_tokens',
                                                            'processed_text']]
XTRAlong_articles_df = pd.DataFrame(XTRAlong_articles_idx)
XTRAlong_articles_df.head()
XTRAlong_articles_df2 = pd.DataFrame(XTRAlong_articles_df.iloc[1,1])

In [ ]:
XTRAlong_articles_df2.shape

In [ ]:
XTRAlong_articles_df2.describe()

In [ ]:
XTRAlong_articles_df2.head(100)

In [ ]:
XTRAlong_articles_df2.loc[2937,'url']

In [ ]:
XTRAlong_articles_df2.loc[2937,'processed_text']

In [ ]:
XTRAlong_articles_df2.loc[130,'url']

In [ ]:
XTRAlong_articles_df2.loc[130,'processed_text']

In [ ]:
XXTRAlong_articles_idx = \
api_data_complete_df.groupby(api_data_complete_df['num_tokens'] >3000)[['url',
                                                            'num_tokens',
                                                            'processed_text']]
XXTRAlong_articles_df = pd.DataFrame(XXTRAlong_articles_idx)
XXTRAlong_articles_df.head()
XXTRAlong_articles_df2 = pd.DataFrame(XXTRAlong_articles_df.iloc[1,1])

In [ ]:
XXTRAlong_articles_df2.shape

In [ ]:
XXTRAlong_articles_df2.describe()

In [ ]:
XXTRAlong_articles_df2.tail(100)

In [ ]:
XXTRAlong_articles_df2.loc[789,'url']

In [ ]:
XXTRAlong_articles_df2.loc[789,'processed_text']

In [ ]:
reviews_articles_idx = \
XXTRAlong_articles_df2.groupby(XXTRAlong_articles_df2['processed_text']\
                                .str.contains('recommendation'))[['url',
                                                              'num_tokens',
                                                              'processed_text']]
reviews_articles_df = pd.DataFrame(reviews_articles_idx)
reviews_articles_df.head()
reviews_articles_df2 = pd.DataFrame(reviews_articles_df.iloc[1,1])

In [ ]:
NONreviews_articles_df2 = pd.DataFrame(reviews_articles_df.iloc[0,1])

In [ ]:
NONreviews_articles_df2.head()

In [ ]:
NONreviews_articles_df2.loc[1335,'url']

In [ ]:
NONreviews_articles_df2.loc[1335,'processed_text']

In [ ]:
XXTRAlong_articles_df2.shape

In [ ]:
reviews_articles_df2.shape

In [ ]:
reviews_articles_df2.tail(100)

----

**Articles that had more than 1000 tokens tended to have a large number of reviews/recommendation of products or interviews. Above 3,000 tokens were almost entirely reviews. Therefore, the extra long articles in the corpus seem to be valid articles without the artifacts of ads or comments**

**Of the articles with more than 3,000 words, which totalled 224, 178 were product recommendations, while others (from what can be gathered) appear to be stories or many mini parts of the same story with 'Click to Read More' links after each one.**

**These longer articles are therefore not outliers of non-article content but a good amount of the super long articles are product recommendations, which may or may not be relevant in this study.**

---

In [ ]:
api_data_complete_df['num_tokens'].median()

In [ ]:
authors = api_data_complete_df['author'].value_counts()
authors.shape

In [ ]:
authors

In [ ]:
api_data_complete_df['author'].mode()

---

**The number of total articles collected are *36,405* and since we found 3 records have missing processed text, we have a total number of records used for this analysis of *36,402*.** 

**Average Token count is *433.7793* words per article, with a median of *336* due to the extremely high counts in the data. The Histogram for the number of Tokens in each article is right skewed, as the difference between mean and medium indicates.**

**The standard deviation (variability) is about as much *415.6439* given that there are a couple records with *0* words per article (were not parsed) and that there are some articles with significantly large token counts up to a maximum of *7240* words.**

**The 75% percentile of 507 indicates most of the data lies in the lower range and just a few articles out of the 36,405 have word counts over 507.**

**The total number of unique authors present in the data are *7,288*, with The Associated Press being the largest presence as 'author' with *1,847* articles attributed to them. This is without counting those with the authorship of "Associated Press" (absent "The"), which account for *504* articles attributed to that authorship.**

**These results show we have gathered an adequate amount of data for our analysis.**

---

### Word Clouds

In [ ]:
api_data_complete_df['tokens'] = \
api_data_complete_df['processed_text'].apply(prepare,
                                             pipeline=tokenize_sw_pipeline)

In [ ]:
Text_Words = [token for sublist in
                api_data_complete_df['tokens'] 
                for token in sublist]
Text_Words_counts = collections.Counter(Text_Words)

In [ ]:
#Text_Words

In [ ]:
print("\nNews articles' top 20 words:\n")
for HT, count in Text_Words_counts.most_common(20):
    print(f"{HT}: {count}")

In [ ]:
Text_Words_counts2 = count_words(Text_Words, split=0)
#display(Text_Words_counts2)

In [ ]:
wordcloud(Text_Words_counts2['freq'],
          title="WordCloud of entire News Data Corpus",
          max_words=500)

---

**With the preprocessing already done, the wordcloud doesn't tell us much. This is because it includes coverage of a variety of topics and news sources for the month of June. A deeper look into topics and sentiment might offer better opportunities to observe things like wordclouds and hence derive meaningful insights**

---

### Topic Modeling Non-Negative Matrix (NMF):

In [ ]:
pipeline_basic = [remove_punctuation,remove_messy]
api_data_complete_df['clean_processed_text'] = \
api_data_complete_df['processed_text'].apply(prepare,
                                             pipeline=pipeline_basic)

In [ ]:
tfidf_topic = TfidfVectorizer(stop_words=list(sw),
                              min_df=5,
                              max_df=0.7,
                              ngram_range=(1,2))

topic_modeling_input = \
tfidf_topic.fit_transform(api_data_complete_df['clean_processed_text'])

In [ ]:
nmf_text_model_news = NMF(n_components=10, random_state=314)
text_matrix = nmf_text_model_news.fit_transform(topic_modeling_input)
H_text_matrix = nmf_text_model_news.components_

Display Topics:

In [ ]:
display_topics(nmf_text_model_news, tfidf_topic.get_feature_names_out())

---

**Topic Modeling using NMF reveals that the top 10 topics present in news article data are:**

* Sports/NBA
* Sports/Baseball
* Amazon/Review
* Presidency/Government
* Business/Work/Travel
* Foreign Affairs/Ukraine/Russia
* Technology/AI regulation
* Titan Implosion
* Economy/Inflation
* Police/Court

**Topics are well defined and clearly distinctive using NMF topic modeling.**

---

### Topic Modeling Latent Dirichlet Allocation (LDA):

In [ ]:
count_text_vectorizer = CountVectorizer(stop_words=list(sw),
                                        min_df=5,
                                        max_df=0.7)

count_text_vectors = count_text_vectorizer\
.fit_transform(api_data_complete_df['clean_processed_text'])
count_text_vectors.shape

In [ ]:
lda_para_model = LatentDirichletAllocation(n_components=10, random_state=42)
W_lda_para_matrix = lda_para_model.fit_transform(count_text_vectors)
H_lda_para_matrix = lda_para_model.components_

In [ ]:
display_topics(lda_para_model, count_text_vectorizer.get_feature_names_out())

In [ ]:
lda_display = pyLDAvis.lda_model.prepare(lda_para_model,
                                         count_text_vectors,
                                         count_text_vectorizer,
                                         sort_topics=False)
pyLDAvis.display(lda_display)

In [ ]:
pyLDAvis.save_html(lda_display, 'lda_capstone1.html')

---

**Topic Modeling using LDA reveals that the top 10 topics present in news article data are:**

* AI/business
* Trump/Presidency
* Rates/Market
* One/People
* Chine/School
* Amazon/Review
* Sports/Draft
* Sports/Season
* Police/Year
* Million/Company

**As we can see, the topics overlap and are not very clearly distinctive when using LDA topic modeling.**

----

### Topic Coherence Score Adaptability

---

An attempt is made here to modify the NMF topic model if we were to run this in real time with every new corpus of articles data. 

The approach is to modify the number of topics depending on Coherence scores that are calculated. In the end, this was not used in the current project, but is available for future work.

---

#### NMF Coherence:

In [ ]:
gensim = [[w for w in re.findall(r'\b\w\w+\b', text.lower()) if w not in sw] for text in api_data_complete_df['clean_processed_text']]

In [ ]:
dict_gensim = Dictionary(gensim)

In [ ]:
bow_gensim = [dict_gensim.doc2bow(word) for word in gensim]
tfIdf_gensim = TfidfModel(bow_gensim)
vectors_gensim = tfIdf_gensim[bow_gensim]
#nmf_gensim=Nmf(topic_modeling_input, num_topics=10, id2word=dict_gensim, kappa=0.1, eval_every=5)

In [ ]:
nmf_gensim = Nmf(vectors_gensim,
                 num_topics=10,
                 id2word=dict_gensim,
                 kappa=0.1,
                 eval_every=5,
                 random_state=314)

In [ ]:
nmf_para_coherence = CoherenceModel(model=nmf_gensim,
                                    texts=gensim,
                                    dictionary=dict_gensim,
                                    coherence='c_v')

nmf_para_coherence_score = nmf_para_coherence.get_coherence()
print(nmf_para_coherence_score)

#### A function to quickly detect the ideal number of topics simply based on coherence scores:

In [ ]:
def find_ideal_topic_number(nmf_model, nmf_coherence, n_topics):
    if (nmf_coherence < 0.69):
        n_topics += n_topics+2
        nmf_model = Nmf(vectors_gensim,
                        num_topics=n_topics,
                        id2word=dict_gensim,
                        kappa=0.1,
                        eval_every=5,
                        random_state=314)
        
        nmf_coherence=nmf_para_coherence.get_coherence()
        if nmf_coherence < 0.69:
            n_topics += n_topics+2
            nmf_model = Nmf(vectors_gensim,
                            num_topics=n_topics,
                            id2word=dict_gensim,
                            kappa=0.1,
                            eval_every=5,
                            random_state=314)
            
            nmf_coherence=nmf_para_coherence.get_coherence()
            if nmf_para_coherence_score < 0.69:
                n_topics += n_topics+2
                nmf_model = Nmf(vectors_gensim,
                                num_topics=n_topics,
                                id2word=dict_gensim,
                                kappa=0.1,
                                eval_every=5,
                                random_state=314)
                
                nmf_coherence = nmf_para_coherence.get_coherence()
            else:
                return(nmf_model, nmf_coherence, n_topics)
        else:
            return(nmf_model, nmf_coherence, n_topics)
    else:
        return(nmf_model, nmf_coherence, n_topics)
    
    

In [ ]:
nmf_gensim, nmf_para_coherence_score, n_topics = \
find_ideal_topic_number(nmf_gensim,
                        nmf_para_coherence_score,
                        10)

### Add NMF topics as features:

In [ ]:
# run the NMF for topic definitions with the current ideal
# value for n_topics derived earlier:
nmf_text_model_news = NMF(n_components=n_topics, random_state=314)
text_matrix = nmf_text_model_news.fit_transform(topic_modeling_input)
H_text_matrix = nmf_text_model_news.components_

In [ ]:
topic_names = []
voc = tfidf_topic.get_feature_names_out()
for topic in nmf_text_model_news.components_:
    important = topic.argsort()
    top_word = voc[important[-1]] + " " + voc[important[-2]]
    topic_names.append(top_word)

Include other topics only if the weight is greater than 0.007

In [ ]:
#topic_ = defaultdict(list)
for idx, row in enumerate(text_matrix) : 
    topic3 = np.where(row == np.amax(row))[0]
    in_order = row.argsort()
    #if (text_matrix[idx][in_order[9]]<0.006):
    #    print(text_matrix[idx][in_order[9]])
    if (text_matrix[idx][in_order[8]]>0.007):
        topic2nd = in_order[8]
    else:
        topic2nd = ""
    if (text_matrix[idx][in_order[7]]>0.007):
        topic3rd =  in_order[7]
    else:
        topic3rd = ""
    important_ = topic3.argsort()
    #print(topic3, topic2nd, topic3rd)
    api_data_complete_df.at[idx,"topic"]=topic3[0]
    api_data_complete_df.at[idx,"topic_name"]=topic_names[topic3[0]]
    if topic2nd != "":
        api_data_complete_df.at[idx,"topic_name_2nd"] = topic_names[topic2nd]
    else:
        api_data_complete_df.at[idx,"topic_name_2nd"] = ""
    if topic3rd != "":    
        api_data_complete_df.at[idx,"topic_name_3rd"] = topic_names[topic3rd]
    else:
        api_data_complete_df.at[idx,"topic_name_3rd"] = ""
    
    #topic_[topic3[0]].

Instead, it may be more effective to use proportions instead of weights to determine if we should include 2nd and 3rd topics of relevance. Use a threshold of 33%

In [ ]:
'''Obtain row-wise proportions for W matrix and convert top 3 topics over a
threshold for each document to a (n, t) binary array, where n is the
number of records and t is the number of topics citation:
https://chat.openai.com/share/c8cbd38a-b024-4915-939d-cf5f9209926b
'''
np.set_printoptions(suppress=True,
                    precision=4)

# Calculate the row totals
row_totals = np.sum(text_matrix, axis=1)

# Divide each element in the array by its corresponding row total
row_proportions = text_matrix / row_totals[:, np.newaxis]


# Define the threshold for selection (0.33 in this case)
threshold = 0.33
n_top = 5

# Create an empty array to store the results
topic = np.zeros((len(row_proportions), 10), dtype=int)

for idx, row in enumerate(row_proportions):
    # Get indices of elements that satisfy the condition: above the threshold
    above_threshold_indices = np.where(row > threshold)[0]
    
    # Sort the indices based on their corresponding values in descending order
    sorted_indices = above_threshold_indices[np.argsort(row[above_threshold_indices])[::-1]]
    
    # Choose the top X indices if available, or fewer if there are fewer elements above the threshold
    top_3_indices = sorted_indices[:min(n_top, len(sorted_indices))]
    
    # Set the corresponding elements in the 'topic' array to 1
    topic[idx, top_3_indices] = 1


api_data_complete_df['multilabel'] = topic.tolist()

out_cols = ['text_id', 'source_name', 'processed_text']

In [ ]:
predefined_topics = ['business',
                     'politics/government',
                     'sports',
                     'shopping',
                     'technology',
                     'public_concerns']

In [ ]:
np.save("../capstone_master_customer_topics.npy", list(predefined_topics))

### Topic Modeling Distribution Charts

In [ ]:
topics_overall = api_data_complete_df['topic_name'].value_counts(normalize=True)
display(topics_overall)

In [ ]:
topics_overall_df = pd.DataFrame(topics_overall)
topics_overall_df.head()

In [ ]:
topics_overall_df['proportion'] = round(topics_overall_df['proportion'],4)
topics_overall_df = topics_overall_df.sort_values('proportion',
                                                  ascending=True)

In [ ]:
#topics_overall.
ax4 = topics_overall_df['proportion'].plot(kind="barh",
                        x='topic_name',
                        rot=0,
                        legend=True,
                        figsize=(8,12),
                        xlabel='Topics',
                        ylabel='Proportion',
                        #='Political_Lean',
                        title='Topics present in News Corpus by Proportions')
ax4.bar_label(ax4.containers[0])

---

**The highest proportion of articles *27.24%* are about the topic "Business Travel" with "Police Court" following at *27.06%* and with the smallest proportion being "Titanic Submersible" at *1.91%***

**Season/Draft (NBA sports) when added to Prop/Runs (Baseball sports), the Sports topic adds up to *20.57%***

---

### Map  Topics to generalized topics (for visualizations):

In [ ]:
predefined_topics = ['business',
                     'politics/government',
                     'sports',
                     'shopping',
                     'technology',
                     'public_concerns']

In [ ]:
np.save("../capstone_master_customer_topics.npy", list(predefined_topics))

In [ ]:
def mapping_topics(article_df):
    customer_topics = []
    mapped_topic = 'misc'
    for items in article_df['topic_name']:
        if (items == 'season draft') | (items == 'prop runs'):
            mapped_topic = 'sports'
        elif (items == 'trump president') | (items == 'russian prigozhin'):
            mapped_topic = 'politics/government'
        elif items == 'amazon review':
            mapped_topic = 'shopping'
        elif (items == 'inflation rates') | (items == 'business work'):
            mapped_topic = 'business'
        elif items == 'ai generative':
            mapped_topic = 'technology'
        elif (items == 'titanic submersible') | (items == 'police court'):
            mapped_topic = 'public_concerns'
        customer_topics.append(mapped_topic)
    article_df['customer_topics'] = customer_topics
    return article_df

In [ ]:
api_data_complete_modeling_df = mapping_topics(api_data_complete_df)

In [ ]:
api_data_complete_modeling_df.head()

#### LDA coherence

In [ ]:
lda_gensim = LdaModel(corpus=bow_gensim,
                      id2word=dict_gensim,
                      alpha='auto',
                      eta='auto',
                      num_topics=10,
                      eval_every=None,
                      random_state=315)

In [ ]:
lda_coherence = CoherenceModel(model=lda_gensim,
                               texts=gensim,
                               dictionary=dict_gensim,
                               coherence='c_v')

lda_coherence_score = lda_coherence.get_coherence()
print(lda_coherence_score)

---

**NMF topic modeling has a coherence score of *0.7506* while LDA topic modeling has a coherence score of *0.5274*.**

**This is no surprised given the clarity of the topics found using NMF versus LDA where there were mixed topics overlapping one another as can be seen in the LDA charts. Since NMF has a much higher coherence score than LDA, we will use NMF topics for our future time-based charts for topic modeling.**

---

#### LDA Coherence Plots:

In [ ]:
lda_model_n=[]
for n in tqdm(range(5,17)):
    lda_model_test = LdaModel(corpus=bow_gensim,
                              id2word=dict_gensim,
                              alpha='auto',
                              eta='auto',
                              num_topics=n,
                              eval_every=None,
                              random_state=315)
    
    lda_coherence_test = CoherenceModel(model=lda_model_test,
                                        texts=gensim,
                                        dictionary=dict_gensim,
                                        coherence='c_v')
    lda_model_n.append((n, lda_model_test, lda_coherence_test.get_coherence()))

In [ ]:
pd.DataFrame(lda_model_n, columns=['n','model','coherence'])\
.set_index('n')[['coherence']].plot(figsize=(16,9),
                                    fontsize=16)
plt.title('Coherence Scores based on Number of Topics using LDA topic modeling',
          fontsize=16)

In [ ]:
nmf_gensim_test = Nmf(vectors_gensim,
                      num_topics=11,
                      id2word=dict_gensim,
                      kappa=0.1,
                      eval_every=5,
                      random_state=314)

In [ ]:
nmf_gensim_test.show_topics()

In [ ]:
nmf_para_coherence_test = CoherenceModel(model=nmf_gensim_test,
                                         texts=gensim,
                                         dictionary=dict_gensim,
                                         coherence='c_v')

nmf_para_coherence_score_test = nmf_para_coherence_test.get_coherence()
print(nmf_para_coherence_score)

#### NMF Coherence plot:

In [ ]:
nmf_model_n_plot=[]
for n in tqdm(range(5,17)):
    nmf_model_test_plot = Nmf(vectors_gensim,
                              num_topics=n,
                              id2word=dict_gensim,
                              kappa=0.1,
                              eval_every=5,
                              random_state=314)
    
    nmf_coherence_test_plot = CoherenceModel(model=nmf_model_test_plot,
                                             texts=gensim, 
                                             dictionary=dict_gensim,
                                             coherence='c_v')
    
    nmf_model_n_plot.append((n,
                             nmf_model_test_plot,
                             nmf_coherence_test_plot.get_coherence()))

In [ ]:
pd.DataFrame(nmf_model_n_plot, columns=['n',
                                        'model',
                                        'coherence'])\
.set_index('n')[['coherence']].plot(figsize=(16,9),
                                    fontsize=16)
plt.title('Coherence Scores based on Number of Topics using NMF topic modeling',
          fontsize=16)

Given that we should not opt for the highest coherence score, but rather, the more reasonable high score, ideal number of topics based on coherence score alone appear to be 11:

In [ ]:
nmf_gensim_test = Nmf(vectors_gensim,
                      num_topics=11,
                      id2word=dict_gensim,
                      kappa=0.1,
                      eval_every=5,
                      random_state=314)

In [ ]:
nmf_model_n_plot[6]

In [ ]:
nmf_gensim_test.show_topics(num_words=5, formatted=True)

Topics are pretty well defined here and quite similar to the NMF model in scikit-learn - although at least 4 different topics are purely sports-related. These results are pretty similar to the results in our scikit-learn NMF model with 10 topics. Therefore, this confirms that the ideal number of topics lies in this region.

---

**An important thing to note: The textbook Blueprints for Text Analytics Using Python by Jens Albrecht et al., warns against using the maximum value that gives the obvious highest coherence score because it is expected that coherence scores will increase as the number of topics increases. Therefore, a high coherence score alone this does not mean that with that number of topics, the topics will be better or more clearly defined.**

**First, we apply coherence score analysis on LDA modeline, which has already been shown to have less well defined topics. Some models were done using LDA with the numbers considered ideal, but were found to have vague topics.**

**Therefore, the same comparison is done for NMF topic modeling**

**We should keep in mind we are using a different NMF algorithm for coherence score analysis. In this case, we are using gensim, while our final model is using scikit-learn. The reason for this is as follows:**

**The coherence scores for gensim NMF topic modeling were calculated and then the ideal number is found. The one that gave the highest coherence score was 15 - with this, the topics were vague and not well defined. Therefore, the second highest score topic number was used: 11. The coherence score for this number of topics was *0.7415*. Here, we found similar topics to that of our scikit-learn NMF model. Yet, there is still some mixing of topics, so it was decided to move forward with the scikit-learn topic model that had better results that could be observed with the human eye.**

**Given that the number of ideal topics for gensim was still 11, and with just slightly differently defined topics, it does appear the models are pretty similar in structure such that the ideal number of topics was between 10 and 11.**

**This confirms in a mathematical manner the decision to move forward with NMF modeling, and with the number of topics.**

----

### Topical Word Cloud Visualizations

#### NMF Word Clouds for 10 topics:

In [ ]:
wordcloud_topics(nmf_text_model_news,
                 tfidf_topic.get_feature_names_out())

#### LDA Word Clouds for 10 topics:

In [ ]:
wordcloud_topics(lda_para_model,
                 count_text_vectorizer.get_feature_names_out())

-----

### Generalized Topic Distribution Chart

In [ ]:
# Overall distribution of customer_topics across the data in proportions:

api_data_complete_modeling_df['customer_topics']\
.value_counts(normalize=True).sort_values().plot(kind="barh",
                 legend=True,
                 figsize=(5,6),
                 title='Topic Distribution Proportions in Training/Test Data')
plt.xticks(rotation=0)

### Topical Time-Based charts:

In [ ]:
df_day1 = api_data_complete_df[['publish_date', 'customer_topics']]
topics_by_date = df_day1.groupby(df_day1['publish_date'])\
.value_counts(normalize=True)

topics_by_date.head()


df_day = pd.DataFrame(topics_by_date)#.set_index(0)
df_day.head(20)
dfg = df_day.groupby(['publish_date',
                      'customer_topics']).agg({'proportion': sum}).reset_index()
dfg.head()
df = dfg.set_index(['publish_date', 'customer_topics']).unstack()
df.plot()
plt.title('Daily Topic Focus by Proportion')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xticks(rotation=45)
plt.figure(figsize=(25, 7))


df.plot.area(colormap="Pastel2")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xticks(rotation=45)
plt.title('Daily Topical Proportions')
plt.figure(figsize=(25, 7))

---

**The topic modeling charts above are generated using Non-Negative Matrix Topic modeling (since this model had more well-defined topics than Latent Dirichlet Allocation modeling) and reveal the most relevant topic in the beginning of June was "Business and Travel," to then taper off to similar levels as that of "Police and Court."**

**Although TV media coverage was overwhelmingly about the titan submersible for that month, the online media does not exhibit this same behavior as it is covered, but not in a larger proportion as other topics.**

**Sports coverage appears to be high in proportion as a topic for the month since the topics "Season and Draft," which is likely about the NBA (after looking at the relevant wordcloud for this topic), and "Prop and Runs," which is likely refering to Baseball both add up to be significant topics in proportion**

----

In [ ]:
topics_overall = api_data_complete_df['topic_name'].value_counts(normalize=True)
display(topics_overall)

In [ ]:
topics_overall_df = pd.DataFrame(topics_overall)
topics_overall_df.head()

In [ ]:
topics_overall_df['proportion'] = round(topics_overall_df['proportion'],4)
topics_overall_df = topics_overall_df.sort_values('proportion', ascending=True)

In [ ]:
#topics_overall.
ax4 = topics_overall_df['proportion'].plot(kind="barh",
                        x='topic_name',
                        rot=0,
                        legend=True,
                        figsize=(8,12),
                        xlabel='Topics',
                        ylabel='Proportion',
                        #='Political_Lean',
                        title='Topics present in News Corpus by Proportions')
ax4.bar_label(ax4.containers[0])

---

**The highest proportion of articles *27.24%* are about the topic "Business Travel" with "Police Court" following at *27.06%* and with the smallest proportion being "Titanic Submersible" at *1.91%***

**Season/Draft (NBA sports) when added to Prop/Runs (Baseball sports), the Sports topic adds up to *20.57%***

---

### Analysis of "Amazon review" Topic

In [ ]:
api_data_complete_df = \
pd.read_csv('../data/capstone_master_3topic_assignment_0725.csv')

In [ ]:
api_data_complete_df.head(20)

In [ ]:
Amazon_articles_idx = \
api_data_complete_df\
.groupby(api_data_complete_df['topic_name'] == 'amazon review')['url']
Amazon_articles_df = pd.DataFrame(Amazon_articles_idx)
Amazon_articles_df.head()
Amazon_articles_df2 = pd.DataFrame(Amazon_articles_df.iloc[1,1])

In [ ]:
Amazon_articles_df2.head()

In [ ]:
Amazon_articles_df2.loc[16,'url']

In [ ]:
Amazon_articles_df2.shape

---

**Upon deeper analysis of the topic_name "amazon review," we find that the total number of articles categorized as this topic are *1,369*. Most of these articles are Review-based on items purchased on Amazon. But some are ads or others are actual stories. Given the small number overall and the difficulty in parsing out the articles that are related to ads or reviews without removing the valid stories, we will keep these articles for now and analyze positive sentiment articles to see if these show up to then filter accordingly.**

---

### Save data for UI:

In [ ]:
api_data_complete_df.to_csv("capstone_master_topic_assignment_0728.csv",
                            sep=',')

In [ ]:
read_back = pd.read_csv("capstone_master_topic_assignment_0728.csv")

In [ ]:
article_data_customer_topic=read_back.drop(['Unnamed: 0',
                                            'index',
                                            'author',
                                            'article_text',
                                            'processed_text',
                                            'clean_processed_text',
                                            'processed_text_split',
                                            'num_tokens',
                                            'original_word_count',
                                            'topic',
                                            'topic_name',
                                            'topic_name_2nd',
                                            'topic_name_3rd'],
                                           axis=1)

In [ ]:
article_data_customer_topic.to_csv("capstone_master_topic_assignment_0729.csv",
                                   sep=',')

----------------
